<a href="https://colab.research.google.com/github/alexmacharia/deep_learning_projects/blob/main/Medical_Images_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [17]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense
from keras.optimizers import Adam

In [9]:
train_path = "/content/gdrive/MyDrive/omdena_project/medical_images/train"
test_path = "/content/gdrive/MyDrive/omdena_project/medical_images/test"
val_path = "/content/gdrive/MyDrive/omdena_project/medical_images/val"


In [10]:
train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=train_path, target_size=(224,224),batch_size=10)
valid_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=val_path, target_size=(224,224),batch_size=10)
test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=test_path, target_size=(224,224),batch_size=10, shuffle=True)

Found 6326 images belonging to 4 classes.
Found 38 images belonging to 4 classes.
Found 771 images belonging to 4 classes.


In [11]:
train_batches

In [15]:
model = Sequential([
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same', input_shape=(224,224,3)),
    MaxPool2D(pool_size=(2,2), strides=2),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
    Flatten(),
    Dense(units=4, activation='softmax')
])

In [19]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics='accuracy')

In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 112, 112, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 112, 112, 64)      18496     
                                                                 
 flatten (Flatten)           (None, 802816)            0         
                                                                 
 dense (Dense)               (None, 4)                 3211268   
                                                                 
Total params: 3,230,660
Trainable params: 3,230,660
Non-trainable params: 0
______________________________________________

In [ ]:
history = model.fit(x=train_batches,
                    steps_per_epoch=len(train_batches),
                    validation_data=valid_batches,
                    validation_steps=len(valid_batches),
                    epochs=10,
                    verbose=2
                    )

Epoch 1/10
633/633 - 3510s - loss: 19.8735 - accuracy: 0.8505 - val_loss: 1.0919 - val_accuracy: 0.6842 - 3510s/epoch - 6s/step
Epoch 2/10
633/633 - 526s - loss: 0.1526 - accuracy: 0.9524 - val_loss: 1.3904 - val_accuracy: 0.6842 - 526s/epoch - 831ms/step
Epoch 3/10
633/633 - 516s - loss: 0.0487 - accuracy: 0.9861 - val_loss: 2.2379 - val_accuracy: 0.6316 - 516s/epoch - 815ms/step
Epoch 4/10
633/633 - 515s - loss: 0.1579 - accuracy: 0.9687 - val_loss: 1.8335 - val_accuracy: 0.6579 - 515s/epoch - 814ms/step
Epoch 5/10
633/633 - 516s - loss: 0.1439 - accuracy: 0.9685 - val_loss: 2.0181 - val_accuracy: 0.6053 - 516s/epoch - 815ms/step
Epoch 6/10
633/633 - 519s - loss: 0.0474 - accuracy: 0.9866 - val_loss: 1.5632 - val_accuracy: 0.6579 - 519s/epoch - 820ms/step
Epoch 7/10
633/633 - 517s - loss: 0.0908 - accuracy: 0.9813 - val_loss: 1.3184 - val_accuracy: 0.7368 - 517s/epoch - 817ms/step
Epoch 8/10
633/633 - 511s - loss: 0.2544 - accuracy: 0.9682 - val_loss: 4.9157 - val_accuracy: 0.5789 - 